# Setup Environment

In [1]:
# To run locally, uncomment the below
# ! pip install -e ../

# To run in Onyx, uncomment the below
import sys
sys.path.append("..")

In [2]:
from onyxgenai.extract import extract_text_from_folder, tokenize_sentences
from onyxgenai.embed import Embedder
from onyxgenai.generate import TextGenerator
from onyxgenai.model import ModelStore

# Extract text from mixed document types

In [3]:
text,files = extract_text_from_folder("../data")
all_sentences = tokenize_sentences(text) 
sentences = [s for ss in all_sentences for s in ss] #flatten

# Embed and persist this data for RAG

In [4]:
embedding_service = Embedder("http://embed.onyx-services/", model="all-MiniLM-L6-v2", model_version=8, num_workers=1, collection_name="jbouder-test")
embeddings= embedding_service.embed_text(sentences)


embedding 55 items
metadata: None


# Start/Serve the Embedding Model

In [3]:
model_service = ModelStore("http://store.onyx-services/")
app_name = model_service.serve_model(model="all-MiniLM-L6-v2", model_version=8, app_name="minilm-jb")
print(app_name)

Deployment Successful: {'message': 'minilm-jb deployment started successfully!'}
minilm-jb


# Embed Prompt for Vector Search

In [4]:
data = ["This is a test sentence.", "Another test sentence for prediction."]
embeddings = model_service.predict_text(app_name=app_name, data=data)
print(embeddings)

Prediction Successful: {'data': ['This is a test sentence.', 'Another test sentence for prediction.'], 'embeddings': [[0.08429651707410812, 0.057953692972660065, 0.004493300803005695, 0.10582106560468674, 0.007083395030349493, -0.01784471981227398, -0.01688806153833866, -0.015228313393890858, 0.04047303646802902, 0.03342254087328911, 0.10432766377925873, -0.04703592136502266, 0.006884709931910038, 0.041017938405275345, 0.01871194876730442, -0.04149230942130089, 0.02364742010831833, -0.05650188401341438, -0.033696312457323074, 0.05099097639322281, 0.06930325925350189, 0.05478424206376076, -0.009788415394723415, 0.02369719371199608, 0.019996581599116325, 0.009717285633087158, -0.0588991716504097, 0.007307434920221567, 0.047026537358760834, -0.004510059021413326, -0.055799730122089386, -0.004159431904554367, 0.06475705653429031, 0.048076264560222626, 0.017020881175994873, -0.0031833869870752096, 0.0574023500084877, 0.03523190692067146, -0.0058839404955506325, 0.01483288872987032, 0.011576

# Perform Vector Search

# Execute a RAG workflow

In [6]:
prompt = "What ticketing tools has DHS utilized for IT services?"
answer = generator_service.generate(prompt,context=embedding_service.query_text(prompt) )


In [7]:
print(answer)

This is sample response from Mistral-Instruct-7B, with prompt of 'What ticketing tools has DHS utilized for IT services?' with context of ['This is result 0', 'This is result 1', 'This is result 2', 'This is result 3', 'This is result 4']


# Shutdown the services

In [6]:
model_service.stop_model(app_name)


Cleanup Successful: {'message': 'minilm-jb deployment cleaned up successfully!'}


{'message': 'minilm-jb deployment cleaned up successfully!'}

# Get List of Available Models

In [17]:
model_service = ModelStore("http://store.onyx-services/")
models = model_service.get_models()
print(models)

{'total_models': 4, 'models': [{'name': 'metaclip-b32-400m', 'versions': [{'version': '5', 'run_id': '220c07a8f6d946659b2d260cff8c7403', 'status': 'READY', 'stage': 'None'}, {'version': '4', 'run_id': 'a89b493232034fbe967c02c714785825', 'status': 'READY', 'stage': 'None'}, {'version': '3', 'run_id': 'bb72496ddb4c42eb812942bbde01a62e', 'status': 'READY', 'stage': 'None'}, {'version': '2', 'run_id': '8ac024607e554d14acdaf6aec2cdd24a', 'status': 'READY', 'stage': 'None'}, {'version': '1', 'run_id': 'dc44896333384c01913937db788a1d87', 'status': 'READY', 'stage': 'None'}]}, {'name': 'dinov2-base', 'versions': [{'version': '4', 'run_id': '75c5fe90554942e0bff11fc1bbbe5d6e', 'status': 'READY', 'stage': 'None'}, {'version': '3', 'run_id': 'fe9723c1d65d4c33b5170f5e8ae64586', 'status': 'READY', 'stage': 'None'}, {'version': '2', 'run_id': 'f3dba1a5c90f4eb6a02e0c1701d39b91', 'status': 'READY', 'stage': 'None'}, {'version': '1', 'run_id': '1fa4620dca844bc582853a22bbc370d1', 'status': 'READY', 'stag